In [1]:
from google.colab import files
uploaded = files.upload()

Saving cleaned_stock_prices.csv to cleaned_stock_prices.csv


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import joblib
import zipfile
from google.colab import files
df = pd.read_csv("cleaned_stock_prices.csv")
target = "Price"
def convert_volume(value):
    if isinstance(value, str):
        value = value.replace(",", "").strip()
        if value.endswith("M"):
            return float(value[:-1]) * 1_000_000
        elif value.endswith("K"):
            return float(value[:-1]) * 1_000
        elif value == "-" or value == "":
            return 0
        else:
            return float(value)
    return value
if "Vol." in df.columns:
    df["Vol."] = df["Vol."].apply(convert_volume)

def clean_percentage(x):
    if isinstance(x, str):
        x = x.replace("%", "").strip()
        return float(x) / 100
    return x
if "Change %" in df.columns:
    df["Change %"] = df["Change %"].apply(clean_percentage)

if "Date" in df.columns:
    df = df.drop(columns=["Date"])
X = df.drop(columns=[target])
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
ann = Sequential([
    Dense(64, activation='relu', input_dim=X_train.shape[1]),
    Dense(32, activation='relu'),
    Dense(1)
])
ann.compile(optimizer='adam', loss='mse')
ann.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, verbose=1)
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
rf = RandomForestRegressor(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
results = pd.DataFrame({
    "Model": ["Linear Regression", "Random Forest", "ANN"],
    "MSE": [
        mean_squared_error(y_test, y_pred_lr),
        mean_squared_error(y_test, y_pred_rf),
        mean_squared_error(y_test, ann.predict(X_test))
    ],
    "R2 Score": [
        r2_score(y_test, y_pred_lr),
        r2_score(y_test, y_pred_rf),
        r2_score(y_test, ann.predict(X_test))
    ]
})
print("\n=== MODEL PERFORMANCE COMPARISON ===")
print(results)


Epoch 1/50


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 174274.0000 - val_loss: 172779.6250
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 174439.6875 - val_loss: 172594.5312
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 174125.7969 - val_loss: 172412.0781
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 173257.7656 - val_loss: 172223.6562
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 175849.1875 - val_loss: 172030.1562
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 174027.5469 - val_loss: 171830.3438
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 173032.6406 - val_loss: 171612.1562
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 173003.3750 - val_loss: 171371.4219
Epoch 9/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 173069.9062 - val_loss: 171105.7031
Epoch 10/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 171793.0312 - val_loss: 170810.4688
Epoch 11/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 173498.0000 - val_loss

In [3]:
with zipfile.ZipFile("regression_outputs.zip", "w") as zipf:
    results.to_csv("regression_model_results.csv", index=False)
    zipf.write("regression_model_results.csv")
    joblib.dump(lr, "linear_regression_model.pkl")
    zipf.write("linear_regression_model.pkl")
    joblib.dump(rf, "random_forest_model.pkl")
    zipf.write("random_forest_model.pkl")
    ann.save("ann_model.h5")
    zipf.write("ann_model.h5")
files.download("regression_outputs.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>